In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import folium # plotting library
from folium import plugins

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns


import os
for dirname, _, filenames in os.walk('/Users/chive/documents/MacAir001STUDY/004Fall2024/CSC7740_BigData/CSC7740final/dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/Users/chive/documents/MacAir001STUDY/004Fall2024/CSC7740_BigData/CSC7740final/dataset/.DS_Store
/Users/chive/documents/MacAir001STUDY/004Fall2024/CSC7740_BigData/CSC7740final/dataset/WeatherEvents_Jan2016-Dec2022.csv
/Users/chive/documents/MacAir001STUDY/004Fall2024/CSC7740_BigData/CSC7740final/dataset/weather_events_demo.csv
/Users/chive/documents/MacAir001STUDY/004Fall2024/CSC7740_BigData/CSC7740final/dataset/final_dataset/.part-00000-e6e632a8-4d8f-4580-8fda-e4d46dd12652-c000.csv.crc
/Users/chive/documents/MacAir001STUDY/004Fall2024/CSC7740_BigData/CSC7740final/dataset/final_dataset/part-00000-e6e632a8-4d8f-4580-8fda-e4d46dd12652-c000.csv
/Users/chive/documents/MacAir001STUDY/004Fall2024/CSC7740_BigData/CSC7740final/dataset/final_dataset/._SUCCESS.crc
/Users/chive/documents/MacAir001STUDY/004Fall2024/CSC7740_BigData/CSC7740final/dataset/final_dataset/_SUCCESS


In [5]:
df = pd.read_csv('/Users/chive/documents/MacAir001STUDY/004Fall2024/CSC7740_BigData/CSC7740final/dataset/WeatherEvents_Jan2016-Dec2022.csv')

df.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Start a Spark session
spark = SparkSession.builder.appName("WeatherDataProcessing").config("spark.driver.memory", "5g").getOrCreate()

# Path to the new dataset
weather_data_path = '/Users/chive/documents/MacAir001STUDY/004Fall2024/CSC7740_BigData/CSC7740final/dataset/WeatherEvents_Jan2016-Dec2022.csv'  # Update this with the correct path on your system

# Read CSV file into Spark DataFrame
weather_data_df = spark.read.csv(weather_data_path, header=True, inferSchema=True)

# Define relevant columns based on the new dataset
DATETIME_START_COL = 'StartTime(UTC)'
DATETIME_END_COL = 'EndTime(UTC)'
EVENT_TYPE_COL = 'Type'
SEVERITY_COL = 'Severity'
PRECIPITATION_COL = 'Precipitation(in)'
AIRPORTCODE_COL= 'AirportCode'
LATITUDE_COL = 'LocationLat'
LONGITUDE_COL = 'LocationLng'
CITY_COL = 'City'
STATE_COL = 'State'

# Select the relevant columns to keep for further processing
weather_data_selected_df = weather_data_df.select(DATETIME_START_COL, DATETIME_END_COL, EVENT_TYPE_COL, SEVERITY_COL, 
                                                  PRECIPITATION_COL, AIRPORTCODE_COL, LATITUDE_COL, LONGITUDE_COL, CITY_COL, STATE_COL)

# Show the first few rows of the full dataset with selected columns
weather_data_selected_df.show()



24/09/26 22:09:02 WARN Utils: Your hostname, CHIVEs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.44 instead (on interface en0)
24/09/26 22:09:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/26 22:09:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------------------+-------------------+----+--------+-----------------+-----------+-----------+-----------+--------+-----+
|     StartTime(UTC)|       EndTime(UTC)|Type|Severity|Precipitation(in)|AirportCode|LocationLat|LocationLng|    City|State|
+-------------------+-------------------+----+--------+-----------------+-----------+-----------+-----------+--------+-----+
|2016-01-06 23:14:00|2016-01-07 00:34:00|Snow|   Light|              0.0|       K04V|    38.0972|  -106.1689|Saguache|   CO|
|2016-01-07 04:14:00|2016-01-07 04:54:00|Snow|   Light|              0.0|       K04V|    38.0972|  -106.1689|Saguache|   CO|
|2016-01-07 05:54:00|2016-01-07 15:34:00|Snow|   Light|             0.03|       K04V|    38.0972|  -106.1689|Saguache|   CO|
|2016-01-08 05:34:00|2016-01-08 05:54:00|Snow|   Light|              0.0|       K04V|    38.0972|  -106.1689|Saguache|   CO|
|2016-01-08 13:54:00|2016-01-08 15:54:00|Snow|   Light|              0.0|       K04V|    38.0972|  -106.1689|Saguache|   CO|


In [8]:
# Count the number of rows and columns in the filtered DataFrame
num_rows = weather_data_selected_df.count()
num_columns = len(weather_data_selected_df.columns)

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)


Number of rows: 8627181
Number of columns: 10


In [9]:
# Print the schema of the entire dataset
weather_data_selected_df.printSchema()


root
 |-- StartTime(UTC): timestamp (nullable = true)
 |-- EndTime(UTC): timestamp (nullable = true)
 |-- Type: string (nullable = true)
 |-- Severity: string (nullable = true)
 |-- Precipitation(in): double (nullable = true)
 |-- AirportCode: string (nullable = true)
 |-- LocationLat: double (nullable = true)
 |-- LocationLng: double (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)



In [10]:
# Count missing values for each column in the full dataset
for c in weather_data_selected_df.columns:
    print(f'Missing values of column `{c}` count: {weather_data_df.where(col(c).isNull()).count()}')


Missing values of column `StartTime(UTC)` count: 0


Missing values of column `EndTime(UTC)` count: 0


Missing values of column `Type` count: 0


Missing values of column `Severity` count: 0


Missing values of column `Precipitation(in)` count: 0


Missing values of column `AirportCode` count: 0


Missing values of column `LocationLat` count: 0


Missing values of column `LocationLng` count: 0


Missing values of column `City` count: 16912


Missing values of column `State` count: 0


In [11]:
# Drop rows with any null values in the full dataset
not_null_weather_data_df = weather_data_selected_df.dropna()

# Show the result after dropping rows with null values
not_null_weather_data_df.show()



+-------------------+-------------------+----+--------+-----------------+-----------+-----------+-----------+--------+-----+
|     StartTime(UTC)|       EndTime(UTC)|Type|Severity|Precipitation(in)|AirportCode|LocationLat|LocationLng|    City|State|
+-------------------+-------------------+----+--------+-----------------+-----------+-----------+-----------+--------+-----+
|2016-01-06 23:14:00|2016-01-07 00:34:00|Snow|   Light|              0.0|       K04V|    38.0972|  -106.1689|Saguache|   CO|
|2016-01-07 04:14:00|2016-01-07 04:54:00|Snow|   Light|              0.0|       K04V|    38.0972|  -106.1689|Saguache|   CO|
|2016-01-07 05:54:00|2016-01-07 15:34:00|Snow|   Light|             0.03|       K04V|    38.0972|  -106.1689|Saguache|   CO|
|2016-01-08 05:34:00|2016-01-08 05:54:00|Snow|   Light|              0.0|       K04V|    38.0972|  -106.1689|Saguache|   CO|
|2016-01-08 13:54:00|2016-01-08 15:54:00|Snow|   Light|              0.0|       K04V|    38.0972|  -106.1689|Saguache|   CO|


In [12]:
# Specific weather columns: Group by 'Type' column and count occurrences in the filtered dataset
not_null_weather_data_df.groupBy(EVENT_TYPE_COL).count().show(truncate=False)


+-------------+-------+
|Type         |count  |
+-------------+-------+
|Cold         |231232 |
|Fog          |2009035|
|Storm        |61096  |
|Precipitation|157036 |
|Hail         |2921   |
|Snow         |1156334|
|Rain         |4992615|
+-------------+-------+



In [13]:
from typing import Dict, Iterable
from pyspark.sql.functions import col

# Weather condition dictionary to map specific data points to broader categories
def get_weather_conditions_aggregation_dict(weather_conditions: Iterable[str]) -> Dict[str, str]:
    
    weather_conditions_dict = dict()
  
    for weather_condition in weather_conditions:
        weather_condition_lowered = weather_condition.lower()

        if any(key in weather_condition_lowered for key in ['squall', 'thunderstorm']):
            weather_conditions_dict[weather_condition] = 'thunderstorm'
        elif any(key in weather_condition_lowered for key in ['drizzle', 'rain']):
            weather_conditions_dict[weather_condition] = 'rainy'
        elif any(key in weather_condition_lowered for key in ['sleet', 'snow']):
            weather_conditions_dict[weather_condition] = 'snowy'
        elif 'cloud' in weather_condition_lowered:
            weather_conditions_dict[weather_condition] = 'cloudy'
        elif any(key in weather_condition_lowered for key in ['fog', 'mist', 'haze']):
            weather_conditions_dict[weather_condition] = 'foggy'
        elif any(key in weather_condition_lowered for key in ['clear', 'sun']):
            weather_conditions_dict[weather_condition] = 'sunny'
        elif 'cold' in weather_condition_lowered:
            weather_conditions_dict[weather_condition] = 'cold'
        elif 'hail' in weather_condition_lowered:
            weather_conditions_dict[weather_condition] = 'hail'
    
    return weather_conditions_dict

# Select distinct weather conditions (event types) from the dataset
weather_conditions_all = not_null_weather_data_df.select(EVENT_TYPE_COL).distinct().rdd.flatMap(lambda x: x).collect()

# Create a dictionary that maps the weather event types to broader categories
weather_conditions_dict = get_weather_conditions_aggregation_dict(weather_conditions_all)

# Broadcast the dictionary for efficient lookup in a distributed environment
broadcast_dict = spark.sparkContext.broadcast(weather_conditions_dict)



In [14]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

# UDF to map weather event types to broad categories
def map_weather_condition(weather_condition):
    return broadcast_dict.value.get(weather_condition, "Unknown")

# Register the UDF to map specific weather conditions
map_weather_condition_udf = udf(map_weather_condition, StringType())

# Apply the UDF to the event type column to create a new column with broad weather conditions
weather_measurements_aggregate_df = not_null_weather_data_df.withColumn(
    "Broad_Weather_Condition",
    map_weather_condition_udf(col(EVENT_TYPE_COL))
)

# Optionally, drop the original specific weather event type column
weather_measurements_aggregate_general_df = weather_measurements_aggregate_df.drop(EVENT_TYPE_COL)




In [15]:
# Not null and consolidated dataframe with broad weather conditions
SLOW_OPERATIONS: bool = True
if SLOW_OPERATIONS: 
    weather_measurements_aggregate_general_df.show()

+-------------------+-------------------+--------+-----------------+-----------+-----------+-----------+--------+-----+-----------------------+
|     StartTime(UTC)|       EndTime(UTC)|Severity|Precipitation(in)|AirportCode|LocationLat|LocationLng|    City|State|Broad_Weather_Condition|
+-------------------+-------------------+--------+-----------------+-----------+-----------+-----------+--------+-----+-----------------------+
|2016-01-06 23:14:00|2016-01-07 00:34:00|   Light|              0.0|       K04V|    38.0972|  -106.1689|Saguache|   CO|                  snowy|
|2016-01-07 04:14:00|2016-01-07 04:54:00|   Light|              0.0|       K04V|    38.0972|  -106.1689|Saguache|   CO|                  snowy|
|2016-01-07 05:54:00|2016-01-07 15:34:00|   Light|             0.03|       K04V|    38.0972|  -106.1689|Saguache|   CO|                  snowy|
|2016-01-08 05:34:00|2016-01-08 05:54:00|   Light|              0.0|       K04V|    38.0972|  -106.1689|Saguache|   CO|                 

In [16]:
# Define the broad weather condition column
BROAD_WEATHER_COL = 'Broad_Weather_Condition'

# Group by the broad weather condition column and count the occurrences
weather_measurements_aggregate_general_df.groupBy(BROAD_WEATHER_COL).count().show(truncate=False)


+-----------------------+-------+
|Broad_Weather_Condition|count  |
+-----------------------+-------+
|rainy                  |4992615|
|snowy                  |1156334|
|Unknown                |218132 |
|cold                   |231232 |
|hail                   |2921   |
|foggy                  |2009035|
+-----------------------+-------+



In [17]:
# Define the broad weather condition column name
BROAD_WEATHER_COL = 'Broad_Weather_Condition'

# Filter out the rows where Broad_Weather_Condition is 'Unknown'
weather_aggregate_filtered_df = weather_measurements_aggregate_general_df.filter(col(BROAD_WEATHER_COL) != "Unknown")




In [18]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

# Define the window specification, ordering by start time in descending order within each broad weather condition
windowSpec = Window.partitionBy(BROAD_WEATHER_COL).orderBy(col(DATETIME_START_COL).desc())

# Add row number to each row within the window
weather_ranked_df = weather_aggregate_filtered_df.withColumn("row_number", row_number().over(windowSpec))

# Filter the ranked dataframe to get the latest 10,000 datasets for each Broad_Weather_Condition
weather_aggregate_10k_df = weather_ranked_df.filter(col("row_number") <= 10000)

# Drop the row_number column
weather_aggregate_final_df = weather_aggregate_10k_df.drop("row_number")

# Show the final dataframe, grouped by Broad_Weather_Condition and showing the count for each category
weather_aggregate_final_df.groupBy(BROAD_WEATHER_COL).count().show(truncate=False)



+-----------------------+-----+
|Broad_Weather_Condition|count|
+-----------------------+-----+
|rainy                  |10000|
|snowy                  |10000|
|cold                   |10000|
|hail                   |2921 |
|foggy                  |10000|
+-----------------------+-----+



In [19]:
#### Statistical distribution of the dataset

In [20]:
# Display summary statistics of the numerical columns in the final dataframe
weather_aggregate_final_df.describe().show()


24/09/26 22:13:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------+-------------------+-----------+-----------------+------------------+----------+-----+-----------------------+
|summary|Severity|  Precipitation(in)|AirportCode|      LocationLat|       LocationLng|      City|State|Broad_Weather_Condition|
+-------+--------+-------------------+-----------+-----------------+------------------+----------+-----+-----------------------+
|  count|   42921|              42921|      42921|            42921|             42921|     42921|42921|                  42921|
|   mean|    NULL|0.02936278278698031|       NULL|39.98051975955862|-96.75347946459746|      NULL| NULL|                   NULL|
| stddev|    NULL|0.12242399472858015|       NULL| 5.36094191369729|15.394954701008139|      NULL| NULL|                   NULL|
|    min|   Heavy|                0.0|       K01M|          24.5571|          -124.555| Abbeville|   AL|                   cold|
|    max|  Severe|               4.87|       KZZV|          48.9402|          -67.7928|Zionsville

In [21]:
# Path where the CSV file will be saved
final_dataset_path = "dataset/updated_final_dataset"

# Save the DataFrame as a single CSV file
(weather_aggregate_final_df
    .coalesce(1)  # Combine all partitions into one
    .write
    .mode("overwrite")  # Overwrite if the file already exists
    .option("header", "true")  # Include header
    .csv(final_dataset_path))

